In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import matplotlib.style as style
style.use('seaborn-whitegrid')

import os
import re
import pickle
from collections import defaultdict, OrderedDict

In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.applications import VGG16
from keras.optimizers import SGD
from PIL import Image
from keras import models, layers

import cv2
import imutils

Using TensorFlow backend.


In [3]:
bg = None

In [4]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [5]:
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (_, cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [ ]:
cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',gray)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [23]:
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

# free up memory
camera.release()
cv2.destroyAllWindows()

In [10]:
num_frames

187

In [9]:
thresholded.shape

(215, 240)

In [1]:
# global variables
bg = None

#-------------------------------------------------------------------------------
# Function - To find the running average over the background
#-------------------------------------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

    #-------------------------------------------------------------------------------
    # Function - To segment the region of hand in the image
    #-------------------------------------------------------------------------------
    def segment(image, threshold=5):
        global bg
        # find the absolute difference between background and current frame
        diff = cv2.absdiff(bg.astype("uint8"), image)

        # threshold the diff image so that we get the foreground
        thresholded = cv2.threshold(diff,
                                    threshold,
                                    255,
                                    cv2.THRESH_BINARY)[1]

        # get the contours in the thresholded image
        (_, cnts, _) = cv2.findContours(thresholded.copy(),
                                        cv2.RETR_EXTERNAL,
                                        cv2.CHAIN_APPROX_SIMPLE)

        # return None, if no contours detected
        if len(cnts) == 0:
            return
        else:
            # based on contour area, get the maximum contour which is the hand
            segmented = max(cnts, key=cv2.contourArea)
            return (thresholded, segmented)

            #-------------------------------------------------------------------------------
            # Main function
            #-------------------------------------------------------------------------------
            if __name__ == "__main__":
                # initialize weight for running average
                aWeight = 0.5

                # get the reference to the webcam
                camera = cv2.VideoCapture(0)

                # region of interest (ROI) coordinates
                top, right, bottom, left = 10, 350, 225, 590

                # initialize num of frames
                num_frames = 90

                # keep looping, until interrupted
                while(True):
                    # get the current frame
                    (grabbed, frame) = camera.read()

                    # resize the frame
                    frame = imutils.resize(frame, width=700)

                    # flip the frame so that it is not the mirror view
                    frame = cv2.flip(frame, 1)

                    # clone the frame
                    clone = frame.copy()

                    # get the height and width of the frame
                    (height, width) = frame.shape[:2]

                    # get the ROI
                    roi = frame[top:bottom, right:left]

                    # convert the roi to grayscale and blur it
                    
                    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                    gray = cv2.GaussianBlur(gray, (7, 7), 0)

                    # to get the background, keep looking till a threshold is reached
                    # so that our running average model gets calibrated
                    if num_frames < 30:
                        run_avg(gray, aWeight)
                    else:
                        # segment the hand region
                        hand = segment(gray)

                        # check whether hand region is segmented
                        if hand is not None:
                            # if yes, unpack the thresholded image and
                            # segmented region
                            (thresholded, segmented) = hand

                            # draw the segmented region and display the frame
                            cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                            cv2.imshow("Thresholded", thresholded)

                    # draw the segmented hand
                    cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

                    # increment the number of frames
                    num_frames += 1

                    # display the frame with segmented hand
                    cv2.imshow("Video Feed", clone)

                    # observe the keypress by the user
                    keypress = cv2.waitKey(1) & 0xFF

                    # if the user pressed "q", then stop looping
                    if keypress == ord("q"):
                        break

            # free up memory
            camera.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [15]:
clone.shape

(393, 700, 3)

In [ ]:

cap = cv2.VideoCapture(0)

fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False, history=500)

while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)
    
    cv2.imshow('frame',fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()


In [8]:
import numpy as np
import cv2
import sys

In [ ]:



# both MOG and MOG2 can be used, with different parameter values
backgroundSubtractor = cv2.BackgroundSubtractorMOG()

#backgroundSubtractor = cv2.BackgroundSubtractorMOG(history=100, nmixtures=5, backgroundRatio=0.7, noiseSigma=0)
#backgroundSubtractor = cv2.BackgroundSubtractorMOG2(history=500, varThreshold=500)
#backgroundSubtractor = cv2.BackgroundSubtractorMOG2()

# apply the algorithm for background images using learning rate > 0
for i in range(1, 16):
    bgImageFile = "data/backgrounds/bg_%03d.jpg" % i
    print "Opening background", bgImageFile
    bg = cv2.imread(bgImageFile)
    backgroundSubtractor.apply(bg, learningRate=0.5)

# apply the algorithm for detection image using learning rate 0
stillFrame = cv2.imread("data/test_bg.jpg")
fgmask = backgroundSubtractor.apply(stillFrame, learningRate=0)

# show both images
cv2.imshow("original", cv2.resize(stillFrame, (0, 0), fx=0.5, fy=0.5))
cv2.imshow("mask", cv2.resize(fgmask, (0, 0), fx=0.5, fy=0.5))
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:


model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(224, 224,1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.load_weights('my_model.h5')

In [6]:
from keras.models import load_model
model = load_model('my_model_2.h5')

#### Functions to convert raw images to arrays, predict with Keras model, and return
#### name of predicted gesture

In [10]:
def get_array_from_single_image(path):
    '''
    Takes in a path to a single image, returns it in an array
    that can be fed into the Keras model.
    '''
    x_data = []
    count = 0
    img = Image.open(path).convert('L')
    img = img.resize((224, 224))
    arr = np.array(img)
    x_data.append(arr) 
    count = count + 1
    return np.array(x_data, dtype = 'float32').reshape((count, 224, 224, 1))

In [20]:
a = get_array_from_single_image('images_to_predict/black_background.jpg')

In [12]:
def predict_single_point(arr):
    '''Takes in properly formatted Keras array,
    returns an array with predicted gesture.
    '''
    return model.predict(arr, batch_size=1, verbose=1)

In [13]:
b = predict_single_point(a)

1/1 [==============================] - 0s 71ms/step


In [14]:
def return_gesture_name(prediction):
    y = model.predict(prediction, batch_size=1, verbose=1)
    return gesture_names_2[np.where(y[0] == max(y[0]))[0][0]]

In [15]:
gesture_names = {0: 'thumb',
                 1: 'palm_moved',
                 2: 'l',
                 3: 'palm',
                 4: 'ok',
                 5: 'down',
                 6: 'index',
                 7: 'c',
                 8: 'fist',
                 9: 'fist_moved'}

gesture_names_2 = {0: '05_thumb',
                 1: '08_palm_moved',
                 2: '02_l',
                 3: '01_palm',
                 4: '07_ok',
                 5: '10_down',
                 6: '06_index',
                 7: '09_c',
                 8: '03_fist',
                 9: '04_fist_moved'}

In [16]:
gesture_names_2[np.where(b[0] == max(b[0]))[0][0]]

'09_c'

In [18]:
return_gesture_name(get_array_from_single_image('images_to_predict/black_background.jpg'))

1/1 [==============================] - 0s 10ms/step


'09_c'

In [46]:
!open images_to_predict/frame_00_06_0016.png